Na podstawie poprzedniego zadania napisz funkcję, która przyjmie nazwę pliku do wygenerowania, dane o nabywcy oraz listę pozycji na fakturze - a następnie wygeneruje i zapisze fakturę w pliku o podanej nazwie.

W pliku `dane.json` znajdziesz dane trzech faktur - za pomocą pętli odczytaj te dane i przekaż do funkcji, aby wygenerować faktury w formie plików PDF. Pliki nazwij kolejno `faktura-1.pdf`, `faktura-2.pdf`, itd.

In [1]:
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors 
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image,PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import json 
pdfmetrics.registerFont(TTFont('Calibri', 'Calibri.ttf'))


styles=getSampleStyleSheet()
right_allign = ParagraphStyle('header',parent=styles['Normal'],fontName="Calibri",fontSize=10, leading = 10, alignment = 2)
left_allign = ParagraphStyle('header',parent=styles['Normal'],fontName="Calibri",fontSize=10, leading = 10, alignment = 0)

In [35]:


with open('dane do faktur.json', encoding='utf-8') as user_file:
    file_contents = user_file.read()

data = json.loads(file_contents)

for i,invoice in enumerate(data["invoices"],start=1):
    filename = f'faktura-{i}.pdf'
    buyer = invoice['buyer']
    date_fv = invoice['date']
    items = invoice['items']
    print(filename, buyer,items, date_fv)


faktura-1.pdf {'name': 'Firma XYZ', 'address': 'Adres firmy XYZ', 'nip': 7564984534, 'regon': 324353453} [{'name': 'Grabki', 'price': 4.5, 'amount': 5}, {'name': 'Konewka', 'price': 15, 'amount': 10}, {'name': 'Łopatka', 'price': 6, 'amount': 5}] 1.12.2020
faktura-2.pdf {'name': 'Firma ABC', 'address': 'Adres firmy ABC', 'nip': 8472058276, 'regon': 437598432} [{'name': 'Lalka', 'price': 39.9, 'amount': 6}, {'name': 'Samochodzik', 'price': 16.5, 'amount': 4}, {'name': 'Książka', 'price': 19.9, 'amount': 10}] 11.11.2020
faktura-3.pdf {'name': 'Firma IJK', 'address': 'Adres firmy IJK', 'nip': 7473092210, 'regon': 687432543} [{'name': 'Karabin', 'price': 25, 'amount': 10}, {'name': 'Kulki', 'price': 2.5, 'amount': 100}] 15.12.2020


In [47]:
invoice['items']
for i,item in enumerate(invoice["items"], start=1):
    print(f'{item["price"]}')

4.5
15
6


In [58]:
def generator_fatur(filename,buyer,date_fv, items):
    
    nazwa_firmy_s = 'Firma XYZ'
    adres_firmy_s = 'Adres firmy' 
    nip_numer_s = '709657623'
    nip_s = f'NIP: {nip_numer_s}'
    regon_numer_s = '70989343'
    regon_s = f'REGON:{regon_numer_s}'


    city = 'Warszawa'
    date = date_fv
    nazwa_firmy_n = buyer["name"]
    adres_firmy_n = buyer["address"]
    nip_numer_n = buyer["nip"]
    nip_n = f'NIP: {nip_numer_n}'
    regon_numer_n = buyer["regon"]
    regon_n = f'REGON:{regon_numer_n}' 


    doc = SimpleDocTemplate(filename,pagesize=A4,rightMargin=72,
                            leftMargin=72,topMargin=72,bottomMargin=18)

    data = [['L.p.','Nazwa towaru/usługi', 'Ilość', 'Cena', 'Cena łączna']]
    total = 0

    for i, item in enumerate(items, start=1):
            cena_laczna = item["amount"]*item["price"]
            total += cena_laczna 
            data.append([i,item["name"],item["amount"],f'{item["price"]} zł',f'{cena_laczna} zł']) 
    
    data.append(['Suma','','','',f'{total} zł'])


    Story = []

    Story.append(Paragraph(f'Miejsce wystawienia: {city}', right_allign))
    Story.append(Paragraph(f'Data wystawienia: {date}',right_allign))
    Story.append(Paragraph(f'Data sprzedaży: {date}',right_allign)) 
    Story.append(Paragraph(f'Data zakończenia: {date}',right_allign))
    Story.append(Spacer(1,12))

    Story.append(Paragraph(f'Sprzedawca',left_allign))
    Story.append(Paragraph(f'{nazwa_firmy_s}',left_allign))
    Story.append(Paragraph(f'{adres_firmy_s}',left_allign))
    Story.append(Paragraph(f'{nip_s}',left_allign))
    Story.append(Paragraph(f'{regon_s}',left_allign))
    Story.append(Spacer(1,12))
    Story.append(Spacer(1,12))

    Story.append(Paragraph(f'Nabywca',left_allign))
    Story.append(Paragraph(f'{nazwa_firmy_n}',left_allign))
    Story.append(Paragraph(f'{adres_firmy_n}',left_allign))
    Story.append(Paragraph(f'{nip_n}',left_allign))
    Story.append(Paragraph(f'{regon_n}',left_allign))
    Story.append(Spacer(1,12))
    Story.append(Spacer(1,12))
    Story.append(Spacer(1,12))
    Story.append(Spacer(1,12))


    ts = TableStyle([
        ('INNERGRID', (0,0), (-1,-1), 0.25, "black"),
        ('BOX', (0,0), (-1,-1), 0.25, "black"),
        ('SPAN',(0,-1),(1,-1)),
        ('FONT',(0,0),(-1,-1),'Calibri')
    ])

    invoice = Table(data, style = ts)

    Story.append(invoice)

    Story.append(Spacer(1,12))
    Story.append(Spacer(1,12))
    Story.append(Spacer(1,12))
    Story.append(Spacer(1,12))
    Story.append(Spacer(1,12))
    Story.append(Spacer(1,12))
    Story.append(Paragraph(f'.....................................',left_allign))
    Story.append(Paragraph(f'Podpis osoby upoważnionej do odbioru',left_allign))

    Story.append(Spacer(1,12))
    Story.append(Paragraph(f'.....................................',left_allign))
    Story.append(Paragraph(f'Podpis osoby upoważnionej do wystawienia',left_allign))

    doc.build(Story)






In [59]:
for i,invoice in enumerate(data["invoices"],start=1):
    filename = f'faktura-{i}.pdf'
    buyer = invoice['buyer']
    date_fv = invoice['date']
    items = invoice['items']
    generator_fatur(filename,buyer,date_fv,items)